In [73]:
import Candlesticks.IdentifyTrend as trend
from Candlesticks.Candlestick import Candlestick
import requests
from datetime import datetime
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf
import time

In [87]:
x = []
y = []
for i in range(1532736000, 1595990660, 86400):
    
    superstring = 'https://finnhub.io/api/v1/crypto/candle?symbol=BINANCE:BTCUSDT&resolution=1&from='+str(i) + '&to='+str(i+86400)+'&token=bseji3frh5rdgbcj0dk0'
    try:
        candles = requests.get(superstring).json()

        if (candles['s'] != 'no_data'):
            for i in range(len(candles["c"])-4):
                #print ("Candlestick At Time: " + datetime.fromtimestamp(candles["t"][i]).strftime('%Y-%m-%d %H:%M:%S')	) 
                candlestick1 = Candlestick(candles["o"][i],candles["c"][i],candles["l"][i],candles["h"][i])
                candlestick2 = Candlestick(candles["o"][i+1],candles["c"][i+1],candles["l"][i+1],candles["h"][i+1])
                candlestick3 = Candlestick(candles["o"][i+2],candles["c"][i+2],candles["l"][i+2],candles["h"][i+2])
                #candlestick4 = Candlestick(candles["o"][i+3],candles["c"][i+3],candles["l"][i+3],candles["h"][i+3])
                #candlestick5 = Candlestick(candles["o"][i+4],candles["c"][i+4],candles["l"][i+4],candles["h"][i+4])
                """if (candlestick2 and candlestick3):
                    if (trend.isBullishEngulfing(candlestick1,candlestick2)):
                        x.append([candlestick1.getProperties() + candlestick2.getProperties()])
                        if candles["o"][i+2] > candles["c"][i+2]:
                            y.append(1)
                        else:
                            y.append(0)"""
                if (candlestick4):
                    if (trend.isBullishHarami(candlestick1, candlestick2, candlestick3)):
                        x.append([candlestick1.getProperties() + candlestick2.getProperties() + candlestick3.getProperties()])
                        if candles["o"][i+4] > candles["c"][i+4]:
                            y.append(1)
                        else:
                            y.append(0)
    except:
        continue
    

In [80]:
x_df = pd.DataFrame(x)
y_df = pd.DataFrame(y)
x_df = x_df[0].apply(pd.Series)
y_df = y_df[0].apply(pd.Series)
x_df = x_df.rename(columns = lambda x : 'tag_' + str(x))
y_df = y_df.rename(columns = lambda x : 'tag_' + str(x))


In [81]:
feature_columns = []
NUMERIC_COLUMNS = ['tag_'+str(i) for i in range(8)]
for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
NUMERIC_COLUMNS

['tag_0', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7']

In [82]:
x_train = x_df.iloc[:x_df.shape[0] *3//4]
y_train = y_df.iloc[:y_df.shape[0] *3//4]
x_test = x_df.iloc[x_df.shape[0] *3//4:]
y_test = y_df.iloc[y_df.shape[0] *3//4:]


In [83]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=64):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(x_train, y_train)
eval_input_fn = make_input_fn(x_test, y_test, num_epochs=100, shuffle=True)

In [84]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

W0728 22:26:35.616787  6348 estimator.py:1846] Using temporary folder as model directory: C:\Users\RYANGU~1\AppData\Local\Temp\tmpxutv27q5
W0728 22:26:35.698382  6348 base_layer_v1.py:1825] Layer linear/linear_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W0728 22:26:37.363324  6348 base_layer_v1.py:1825] Layer linear/linear_model is casting an input tensor fr

In [85]:
result

{'accuracy': 0.4090909,
 'accuracy_baseline': 0.59090906,
 'auc': 0.5,
 'auc_precision_recall': 0.4090909,
 'average_loss': 1266.6488,
 'label/mean': 0.4090909,
 'loss': 1266.6489,
 'precision': 0.4090909,
 'prediction/mean': 1.0,
 'recall': 1.0,
 'global_step': 20}